In [ ]:
import time, socket, sys, pickle
import random
from sympy import *

def coprime(a, b):
    while b != 0:
        a, b = b, a % b
    return a
    
    
def extended_gcd(aa, bb):
    lastremainder, remainder = abs(aa), abs(bb)
    x, lastx, y, lasty = 0, 1, 1, 0
    while remainder:
        lastremainder,(quotient,remainder)=remainder,divmod(lastremainder,remainder)
        x, lastx = lastx - quotient*x, x
        y, lasty = lasty - quotient*y, y
    return lastremainder,lastx*(-1 if aa < 0 else 1),lasty*(-1 if bb < 0 else 1)

#Euclid's extended algorithm for finding the multiplicative inverse of two numbers    
def modinv(a, m):
	g, x, y = extended_gcd(a, m)
	if g != 1:
		raise Exception('Modular inverse does not exist')
	return x % m    


def generate_keypair():
    while(True):
        p=random.randint(100, 200)
        if(isprime(p)):
            break;
    
    #picking random primes such that p!=q 
    while(True):
        while(True):
            q=random.randint(100, 200)
            if(isprime(q)):
                break;
        if(p!=q):
            break;
    n = p * q
    phi = (p-1) * (q-1)

    #Choose an integer e such that e and phi(n) are coprime
    while(True):
        e = random.randrange(1, phi)
        g = coprime(e, phi)
        if(g==1):
            break;

    #Use Extended Euclid's Algorithm to generate the private key
    d = modinv(e, phi)
    #Return public and private keypair
    return ((e, n), (d, n))


def encrypt(privatek, plaintext):
    key, n = privatek
    #Convert each letter in the plaintext to numbers based on the character
    numberRepr = [ord(char) for char in plaintext]
    print("Number representation of message before signature generation: "\
          , numberRepr)
    cipher = [pow(ord(char),key,n) for char in plaintext]
    print("Number representation of signature: ", cipher)

    
    return cipher
def decrypt(publick, ciphertext):
    key, n = publick
    #Generate the plaintext based on the ciphertext and key
    numberRepr = [pow(char, key, n) for char in ciphertext]
    plain = [chr(pow(char, key, n)) for char in ciphertext]
    print("Decrypted signature's number representation is: ", numberRepr)
    return ''.join(plain)
#creating a socket
new_socket = socket.socket()
host_name = socket.gethostname()
s_ip = socket.gethostbyname(host_name)
 
port = 5000
 
new_socket.bind((host_name, port))
print("Binding successful!")
print("This is your IP: ", s_ip)
 
name = input('Enter your name: ')
 
new_socket.listen(1) 
 
 
conn, add = new_socket.accept()
 
print("Received connection from ", add[0])
print('Connection Established. Connected From: ',add[0])
 
client = (conn.recv(1024)).decode()
print(client + ' has connected.')
 
conn.send(name.encode())
print("Generating public and private keypairs of",name,"...")
public, private = generate_keypair()
e,n=public
d,n=private
print("Public key of",name," is :("+str(e)+","+str(n)+ ") "\
      "Private key of",name," is :("+str(d)+","+str(n)+")")
print("Sending public key of",name,"to",client,"...")
conn.send(str(e).encode())
conn.send(str(n).encode())
print("sent public key.")
e1=int((conn.recv(1024)).decode())
n1=int((conn.recv(1024)).decode())
public1=(e1,n1)
print("recived the public key of",client," :("+str(e1)+","+str(n1)+")")
while True:  
    message = input('Me : ')
    print("generating signature with",name,"'s private key :", private ," . . .")
    sign_msg = encrypt(private, message)
    conn.send(message.encode())
    conn.send(pickle.dumps(sign_msg))
    print("message,signature sent.")
    message = (conn.recv(1024)).decode()
    sign = pickle.loads((conn.recv(1024)))
    print(client, " message:", message)
    print(client, " signature:", sign)
    print("verifing the signature using",client,\
          "'s public key :(" +str(e1)+","+str(n1)+")")
    m1 = decrypt(public1, sign)
    if(m1==message):
        print("Verified signature,Message is from ",client)
    

Binding successful!
This is your IP:  192.168.1.34
Enter your name: Alice
Received connection from  192.168.1.34
Connection Established. Connected From:  192.168.1.34
Bob has connected.
Generating public and private keypairs of Alice ...
Public key of Alice  is :(11573,19291) Private key of Alice  is :(637,19291)
Sending public key of Alice to Bob ...
sent public key.
recived the public key of Bob  :(5351,17399)
Me : HI,this is alice
generating signature with Alice 's private key : (637, 19291)  . . .
Number representation of message before signature generation:  [72, 73, 44, 116, 104, 105, 115, 32, 105, 115, 32, 97, 108, 105, 99, 101]
Number representation of signature:  [5832, 175, 4412, 5829, 4436, 18276, 15156, 17535, 18276, 15156, 17535, 16872, 10539, 18276, 13075, 15049]
message,signature sent.
Bob  message: hi, i am bob
Bob  signature: [6518, 7390, 2534, 10557, 7390, 10557, 464, 15205, 10557, 4941, 9715, 4941]
verifing the signature using Bob 's public key :(5351,17399)
Decrypte